# Test code and scratch space for MR preproc dashboard 

1. Check output directories to see output files 
2. Check logs to find out issues 

**Expected use case:** 
    Run this script after preprocessing is complete to provide info on each subject

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import re
sys.path.append('../')
from lib.preproc_checks import *
from lib.minc_wrap import *

# Data paths

In [2]:
data_dir = '/Users/nikhil/projects/MR_preproc_dash/data/'
preproc_pipeline_dir = '/Users/nikhil/code/git_repos/nist_mni_pipelines/'

if preproc_pipeline_dir not in sys.path:
    sys.path.append(preproc_pipeline_dir)
    
local_env = '/opt/minc/1.9.16/minc-toolkit-config.sh' 
script_dir = '/Users/nikhil/code/git_repos/MR_preproc_dash/scripts/'

# Expected output dirs

In [3]:
# First level output subdirs
output_dirs = ['clp','clp2','stx','stx2','vbm','cls','vol','lng'] # In each timepoint

# List of files expected in each subdir
task_file_names_dict = {}
task_file_names_dict['clp'] = ['clp','den','nuc'] #_t1
task_file_names_dict['clp2'] = ['clp2'] #_t1
task_file_names_dict['cls'] = ['cls','lob'] 
task_file_names_dict['stx'] = ['stx','nsstx'] #_t1
task_file_names_dict['stx2'] = ['stx2'] #_t1
task_file_names_dict['vol'] = ['vol'] # text file
task_file_names_dict['vbm'] = ['vbm_imp_csf','vbm_imp_gm','vbm_imp_wm']
task_file_names_dict['lng'] = ['lng'] #_t1

# Step 1: Find all the subject subdirectories

In [6]:
# iterate thru all subjects 
pattern = re.compile("([0-9]*_S_[0-9]*)")
subject_names = next(os.walk(data_dir))[1]
df_preproc = pd.DataFrame()
for subject_name in subject_names:
    if not pattern.match(subject_name):
        print('\ndirectory name {} does not match subject naming convention'.format(subject_name))
    else:
        subject_dir = data_dir + subject_name + '/'
        pipeline_data_pickle = pd.read_pickle(subject_dir + '{}.pickle'.format(subject_name))

        # Step1: read the pickle to get subject specific parameters and preproc stages
        df = parse_pickle(pipeline_data_pickle,output_dirs)

        # Step 2: check output directory tree and files 
        df, missing_tp, missing_dir = check_output_dirs(df,output_dirs,subject_dir)
        df, missing_file, reg_param_flat_tp, reg_param_list_tp = check_output_files(local_env,df,task_file_names_dict,subject_dir,script_dir)

        # preproc success/failure summary
        print('')
        print('---------------------------------------------------------------')
        print('subject: {}'.format(df['subject_idx'].values[0]))
        print('missing timepoints (# {}): \n{}'.format(len(missing_tp),missing_tp))
        print('')
        print('missing dirs (# {}): \n{}'.format(len(missing_dir), missing_dir)) 
        print('')
        print('missing files(# {}): \n{}'.format(len(missing_file),missing_file))

        df_preproc = df_preproc.append(df)

print('')
print('Lenth of df_preproc: {}'.format(len(df_preproc)))
print('number of subjects: {}'.format(len(set(df_preproc['subject_idx'].values))))


---------------------------------------------------------------
subject: 052_S_4807
missing timepoints (# 1): 
['20120727']

missing dirs (# 3): 
['20150804/stx', '20150804/vbm', '20150804/cls']

missing files(# 2): 
['20150804/clp/clp_052_S_4807_20150804_t1.mnc', '20150804/clp/nuc_052_S_4807_20150804_t1.mnc']

directory name bad_S_naming does not match subject naming convention

Lenth of df_preproc: 6
number of subjects: 1


# Print color-coded tables

In [5]:
# Will not show colors on github pages
styled_df = df[['subject_idx','tp_idx','qc'] + output_dirs]
styled_df.style.\
    applymap(color_missing_dir).\
    apply(highlight_missing_tp)


,subject_idx,tp_idx,qc,clp,clp2,stx,stx2,vbm,cls,vol,lng
0,052_S_4807,20121019,qc_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists
1,052_S_4807,20120727,qc_exists,timepoint_missing,timepoint_missing,timepoint_missing,timepoint_missing,timepoint_missing,timepoint_missing,timepoint_missing,timepoint_missing
2,052_S_4807,20130215,qc_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists
3,052_S_4807,20150804,qc_exists,file_missing,file_exists,dir_missing,file_exists,dir_missing,dir_missing,file_exists,file_exists
4,052_S_4807,20140807,qc_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists
5,052_S_4807,20130813,qc_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists,file_exists
